In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pylab as plt

import sys
sys.path.append('..')

import tidy3d as td
import tidy3d.web as web

In [2]:
# set up parameters of simulation
grid_xyz = td.GridSpec1DSize(dl=0.01)
pml = td.PMLLayer(profile='standard', num_layers=10)
sim_size = [4, 4, 4]
freq0 = 3e14
fwidth = 1e13
run_time = 10/fwidth

# create structure
dielectric = td.nk_to_medium(n=2, k=0, freq=freq0)
square = td.Structure(
    geometry=td.Box(center=[0, 0, 0], size=[1.5, 1.5, 1.5]),
    medium=dielectric)

# create source
source = td.VolumeSource(
    center=(-1.5, 0, 0),
    size=(0, 0.4, 0.4),
    source_time = td.GaussianPulse(
        freq0=freq0,
        fwidth=fwidth),
    polarization='Jx')

# create monitor
monitor = td.FieldMonitor(
    fields=['Ex', 'Hy'],
    center=(0, 0, 0),
    size=(4, 4, 0),
    freqs=[freq0])

# Initialize simulation
sim = td.Simulation(size=sim_size,
                    grid=td.GridSpec(x=grid_xyz, y=grid_xyz, z=grid_xyz),
                    structures=[square],
                    sources={'source': source},
                    monitors={'monitor': monitor},
                    run_time=run_time,
                    pml_layers=(pml, pml, pml))

task_id = web.upload(sim, task_name='quickstart')
web.start(task_id)
web.monitor(task_id)

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /var/folders/jx/9y0mtn3s3zzb6mzgmsw6s6gr0000gn/T/ipykernel_77116/383648472.py:40 in       │
│ <module>                                                                                  │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/jx/9y0mtn3s3zzb6mzgmsw6s6gr0000gn/T/ipykernel_77116/383648472.py'           │
│ /Users/twhughes/Documents/Flexcompute/Tidy3D-client-revamp/notebooks/../tidy3d/web/webapi │
│ .py:77 in upload                                                                          │
│                                                                                           │
│    74 │   TaskId                                                                          │
│    75 │   │   Unique identifier of task on server.                                        │
│    76 │   """                                                                             │
│ ❱  77 │   return _upload_task(simulation=simulation, task_name=task_name, folder_name=fol │
│    78                                                                                     │
│    79                                                                                     │
│    80 def get_info(task_id: TaskId) -> TaskInfo:                                          │
│                                                                                           │
│ /Users/twhughes/Documents/Flexcompute/Tidy3D-client-revamp/notebooks/../tidy3d/web/webapi │
│ .py:312 in _upload_task                                                                   │
│                                                                                           │
│   309 │   """upload with all kwargs exposed"""                                            │
│   310 │                                                                                   │
│   311 │   # convert to old json and get string version                                    │
│ ❱ 312 │   sim_dict = export_old_json(simulation)                                          │
│   313 │   json_string = json.dumps(sim_dict, indent=4)                                    │
│   314 │                                                                                   │
│   315 │   # TODO: remove node size, time steps, compute weight, worker group              │
│                                                                                           │
│ /Users/twhughes/Documents/Flexcompute/Tidy3D-client-revamp/notebooks/../tidy3d_core/conve │
│ rt.py:367 in export_old_json                                                              │
│                                                                                           │
│   364 │   """Write a Simulation in the old Tidy3D json format."""                         │
│   365 │                                                                                   │
│   366 │   sim_dict = {}                                                                   │
│ ❱ 367 │   sim_dict["parameters"] = old_json_parameters(sim)                               │
│   368 │   medium_list, struct_list = old_json_structures(sim)                             │
│   369 │   sim_dict["materials"] = medium_list                                             │
│   370 │   sim_dict["structures"] = struct_list                                            │
│                                                                                           │
│ /Users/twhughes/Documents/Flexcompute/Tidy3D-client-revamp/notebooks/../tidy3d_core/conve │
│ rt.py:24 in old_json_parameters                                                           │
│                                                   

In [ ]:
web.download(task_id, simulation=sim, path='data/sim_data.hdf5')

In [ ]:
sim_data = web.load_data(task_id, simulation=sim, path='data/sim_data.hdf5')
ax = sim_data['monitor'].Ex.isel(f=0, z=0).imag.plot.pcolormesh(x='x', y='y', vmin=-5e-13, vmax=5e-13, cmap='RdBu')

plt.show()

In [ ]:
sim_data.log